In [ ]:
from datacube.utils.dask import start_local_dask
from dep_tools.loaders import Sentinel2OdcLoader
from dep_tools.namers import LocalPath
from dep_tools.writers import LocalDsWriter
from pystac import Item

from src.grid import grid
from src.run_task import GeoMADSentinelProcessor

from odc.geo.gridspec import GridSpec
from odc.geo import XY
import geopandas as gpd

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Optionally set up a local dask cluster
client = start_local_dask()
client.dashboard_link

In [ ]:
# Study site configuration
country_code = "PG"
region_index = "63,20"
datetime = "2022"
dep_path = LocalPath(
    local_folder="data",
    sensor="ls",
    dataset_id="geomad",
    version="0.0.0",
    time=datetime,
)
item_id = f"{country_code}_{region_index}"

# Set up a data loader
loader = Sentinel2OdcLoader(
    epsg=3832,
    datetime=datetime,
    dask_chunksize=dict(time=1, x=4096, y=4096),
    odc_load_kwargs=dict(
        fail_on_error=False,
        resolution=10,
        # bands=["qa_pixel", "red", "green", "blue", "nir08", "swir16", "swir22"],
        bands=["SCL", "red", "green", "blue"]
    ),
    nodata_value=0,
    keep_ints=True,
    flat_array=True,
)

# And a data processor
processor = GeoMADSentinelProcessor(
    scale_and_offset=False,
    dilate_mask=True,
    work_chunks=(3200, 3200),
    num_threads=10,
    keep_ints=True
)

# And get the study site
pacific_gridspec = GridSpec(crs="EPSG:3832", tile_shape=(3200,3200), resolution=30, origin=XY(-3000000,-4000000))
geobox = pacific_gridspec.tile_geobox(XY(*map(float, region_index.split(","))))

cell = gpd.GeoDataFrame(geometry=[geobox.geographic_extent], crs=4326)

In [ ]:
# Run the load process, which is lazy-loaded
input_data = loader.load(cell)
input_data

In [ ]:
# Plan the processing. Still lazy-loaded
output_data = processor.process(input_data)
output_data

In [ ]:
loaded = output_data.compute()

In [ ]:
loaded[["red", "green", "blue"]].to_array().plot.imshow(robust=True, figsize=(10, 10))

In [ ]:
writer = LocalDsWriter(itempath=dep_path)
out_files = writer.write(output_data, item_id)

In [ ]:
# Make sure written files are readable
stac_path = writer.itempath.path(item_id, ext=".stac-item.json")

item = Item.from_file(stac_path)
item.validate()